# 140.model save / load

### 생성한 모델을 disk 에 저장하고 복원

- model 의 architecture 저장 : json, yaml format  

- model 의 weigth 저장 : h5 file format  

- hdf5(Hierarchical Data Format version 5)는 대용량 데이터를 저장하기 위한 파일 포맷

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras import utils

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

X_train = X_train[:, :, :, np.newaxis]
X_test   = X_test[:, :, :, np.newaxis]
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

# LeNet
model = tf.keras.Sequential()

model.add(Conv2D(6, kernel_size=5, padding="same", input_shape=(28, 28, 1)))
model.add(Activation("relu"))

model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding="valid"))

model.add(Conv2D(16, kernel_size=5, padding="valid"))
model.add(Activation('relu'))

model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))

model.add(Flatten())

model.add(Dense(120))
model.add(Activation("relu"))

model.add(Dense(84))
model.add(Activation("relu"))

model.add(Dense(10))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=128, epochs=1, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 9s 146us/sample - loss: 0.2587 - accuracy: 0.9216 - val_loss: 0.0789 - val_accuracy: 0.9764


### Model Architecture Save

In [2]:
json_string = model.to_json()

In [3]:
with open("lenet.json", "w") as json_file:
    json_file.write(json_string)

### weight 만 save

In [4]:
model.save_weights("lenet_weight.h5")

### 전체 model save

In [5]:
model.save("lenet_model.h5")

## Model Load

- model architecture 와 weight 별도 저장한 경우,  architecture load + weight load + compile

In [6]:
from tensorflow.keras.models import model_from_json

json_file = open("lenet.json", "r")
read_model = json_file.read()
json_file.close()

loaded_model = model_from_json(read_model)
loaded_model.load_weights("lenet_weight.h5")

loaded_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [7]:
loaded_model.evaluate(X_test, y_test, verbose=0)

[0.07888806907087564, 0.9764]

- model 전체 저장한 경우 load

In [8]:
from tensorflow.keras.models import load_model

loaded_model = load_model('lenet_model.h5')

In [9]:
loaded_model.evaluate(X_test, y_test, verbose=0)

[0.07888806907087564, 0.9764]

In [10]:
y_predict = model.predict_classes(X_test)
y_predict

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [11]:
np.argmax(y_test, axis=1)

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [12]:
print("accuracy = {} %".format(np.sum(y_predict == np.argmax(y_test, axis=1)) / len(y_test) * 100))

accuracy = 97.64 %
